### Reduce item_type_code_name levels

In [121]:
%%capture

%run '../lib/init.ipynb'

import pandas as pd
import numpy as np

from ..lib import utilities as util

In [117]:
filterDf  = util.load_df('filterDf')
types = util.get_var_types()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 589, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 254, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/Users/duane.hinkley/PycharmProjects/c772-capstone-project/venv/lib/python3.6/site-packages/pyspark/pyt

###### Display levels

In [50]:
util.count_values(filterDf, 'item_type_code_name').show(50, False)

+----------------------+-----+
|item_type_code_name   |count|
+----------------------+-----+
|multipleChoice        |26704|
|fillInTheBlank        |10497|
|equationEntry         |7841 |
|trueFalse             |4561 |
|cloze                 |3024 |
|multipleSelect        |2295 |
|graphing              |2076 |
|MultipleChoiceResponse|1550 |
|choiceMatrix          |1203 |
|matching              |924  |
|shortAnswer           |853  |
|selectText            |813  |
|bucketing             |799  |
|sortable              |549  |
|essay                 |531  |
|numberLine            |302  |
|aheAlgo               |219  |
|imageLabel            |80   |
|RubricResponse        |11   |
|FillinBlankResponse   |5    |
|fileUpload            |1    |
+----------------------+-----+



###### Display item_type_code_name and scoring_type_code Corelation

In [51]:
dfPd = filterDf.select('item_type_code_name','scoring_type_code').toPandas()
pd.crosstab(dfPd.item_type_code_name.fillna('null'), dfPd.scoring_type_code.fillna('null'), margins=True, margins_name="Total")

scoring_type_code,[unassigned],automatic,external,manual,Total
item_type_code_name,,,,,
FillinBlankResponse,5,0,0,0,5
MultipleChoiceResponse,1550,0,0,0,1550
RubricResponse,11,0,0,0,11
aheAlgo,0,0,219,0,219
bucketing,0,799,0,0,799
choiceMatrix,0,1203,0,0,1203
cloze,0,3024,0,0,3024
equationEntry,0,7841,0,0,7841
essay,9,0,0,522,531


- unassigned
    - FillinBlankResponse
    - MultipleChoiceResponse
    - shortAnswer
- manual
  - aheAlg

In [52]:
util.encode_categories_as_swoe(filterDf).select('item_type_code_name', 'item_type_code_name_swoe').show()

+-------------------+------------------------+
|item_type_code_name|item_type_code_name_swoe|
+-------------------+------------------------+
|         fileUpload|       0.958169577610213|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equati

In [60]:
def swoe2(df):
    # Ref: https://www.researchgate.net/profile/Goutam_Chakraborty4/publication/259184404_Extension_Node_to_the_Rescue_of_the_Curse_of_Dimensionality_via_Weight_of_Evidence_WOE_Recoding/links/0c96052a3a083b6ccd000000.pdf

    # Smoothing parameter.  large = aggressive, small = sensitive to data
    c = 24
    # Get portion of events
    p1 = df.select('target').toPandas().target.mean()

    # Get event and non event counts
    cntDf =  df.groupBy('cat').agg(
        F.sum( F.when( F.col('target')  > 0, 1 ).otherwise(0)).alias('1'),
        F.sum( F.when( F.col('target') == 0, 1 ).otherwise(0)).alias('0')
    )
    # Add swoe
    # Formula:
    #           # events + cp1
    # ln( ----------------------- )
    #     # nonevents + c(1 - p1)
    cntDf = cntDf.withColumn(
        'cat_swoe',
        F.log( (F.col('1') +  c * p1)  / (F.col('0') + c * (1 - p1)))
    )
    return df.join(cntDf.select('cat', 'cat_swoe'), 'cat')

+---+----+-------------------+------------------+
|  0|   1|                woe|                iv|
+---+----+-------------------+------------------+
|0.6|0.25|-0.8754687373538999|0.5924584552018219|
|0.2| 0.5| 0.9162907318741551|0.5924584552018219|
|0.2|0.25|0.22314355131420976|0.5924584552018219|
+---+----+-------------------+------------------+



In [71]:
df = pd.DataFrame(
    {'cat': ['a', 'b', 'a', 'b', 'a', 'a', 'b', 'c', 'c'],
     'target': [1, 0, 0, 1, 0, 0, 1, 1, 0]
     })

feature,target = 'cat','target'
df_woe_iv = (pd.crosstab(df[feature],df[target],
                      normalize='columns')
             .assign(woe=lambda dfx: np.log(dfx[1] / dfx[0]))
             .assign(iv=lambda dfx: np.sum(dfx['woe']*
                                           (dfx[1]-dfx[0]))))

In [101]:
df_woe_iv

target,0,1,woe,iv
cat,,,,
a,0.6,0.25,-0.875469,0.592458
b,0.2,0.50,0.916291,0.592458
c,0.2,0.25,0.223144,0.592458


In [94]:
import math

from pyspark.sql import DataFrame
from pyspark.sql import functions as F

# Ref: https://github.com/albertusk95/weight-of-evidence-spark

class WOE_IV(object):
    def __init__(self, df: DataFrame, cols_to_woe: [str], label_column: str, good_label: str):
        self.df = df
        self.cols_to_woe = cols_to_woe
        self.label_column = label_column
        self.good_label = good_label
        self.fit_data = {}

    def fit(self):
        for col_to_woe in self.cols_to_woe:
            total_good = self.compute_total_amount_of_good()
            total_bad = self.compute_total_amount_of_bad()

            woe_df = self.df.select(col_to_woe)
            categories = woe_df.distinct().collect()
            for category_row in categories:
                category = category_row[col_to_woe]
                good_amount = self.compute_good_amount(col_to_woe, category)
                bad_amount = self.compute_bad_amount(col_to_woe, category)

                good_amount = good_amount if good_amount != 0 else 0.5
                bad_amount = bad_amount if bad_amount != 0 else 0.5

                good_dist = good_amount / total_good
                bad_dist = bad_amount / total_bad

                self.build_fit_data(col_to_woe, category, good_dist, bad_dist)

    def transform(self, df: DataFrame):
        def _encode_woe(col_to_woe_):
            return F.coalesce(
                *[F.when(F.col(col_to_woe_) == category, F.lit(woe_iv['woe']))
                  for category, woe_iv in self.fit_data[col_to_woe_].items()]
            )

        for col_to_woe, woe_info in self.fit_data.items():
            df = df.withColumn(col_to_woe + '_woe', _encode_woe(col_to_woe))
        return df

    def compute_total_amount_of_good(self):
        return self.df.select(self.label_column).filter(F.col(self.label_column) == self.good_label).count()

    def compute_total_amount_of_bad(self):
        return self.df.select(self.label_column).filter(F.col(self.label_column) != self.good_label).count()

    def compute_good_amount(self, col_to_woe: str, category: str):
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (F.col(col_to_woe) == category) & (F.col(self.label_column) == self.good_label)
                      ).count()

    def compute_bad_amount(self, col_to_woe: str, category: str):
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (F.col(col_to_woe) == category) & (F.col(self.label_column) != self.good_label)
                      ).count()

    def build_fit_data(self, col_to_woe, category, good_dist, bad_dist):
        woe_info = {
            category: {
                'woe': math.log(good_dist / bad_dist),
                'iv': (good_dist - bad_dist) * math.log(good_dist / bad_dist)
            }
        }

        if col_to_woe not in self.fit_data:
            self.fit_data[col_to_woe] = woe_info
        else:
            self.fit_data[col_to_woe].update(woe_info)

    def compute_iv(self):
        iv_dict = {}

        for woe_col, categories in self.fit_data.items():
            iv_dict[woe_col] = 0
            for category, woe_iv in categories.items():
                iv_dict[woe_col] += woe_iv['iv']
        return iv_dict

In [95]:
df = pd.DataFrame(
    {'cat': ['a', 'b', 'a', 'b', 'a', 'a', 'b', 'c', 'c'],
     'target': [1, 0, 0, 1, 0, 0, 1, 1, 0]
     })
spDf = spark.createDataFrame(df)

spDf.show()

+---+------+
|cat|target|
+---+------+
|  a|     1|
|  b|     0|
|  a|     0|
|  b|     1|
|  a|     0|
|  a|     0|
|  b|     1|
|  c|     1|
|  c|     0|
+---+------+



In [104]:
cols_to_woe = ['cat']
label_col = 'target'
good_label = 1

woe = WOE_IV(spDf, cols_to_woe, label_col, good_label)

In [106]:
woe.fit()

In [102]:
df_woe_iv

target,0,1,woe,iv
cat,,,,
a,0.6,0.25,-0.875469,0.592458
b,0.2,0.50,0.916291,0.592458
c,0.2,0.25,0.223144,0.592458


In [107]:
woe.transform(spDf).show()

+---+------+-------------------+
|cat|target|            cat_woe|
+---+------+-------------------+
|  a|     1|-0.8754687373538999|
|  b|     0| 0.9162907318741551|
|  a|     0|-0.8754687373538999|
|  b|     1| 0.9162907318741551|
|  a|     0|-0.8754687373538999|
|  a|     0|-0.8754687373538999|
|  b|     1| 0.9162907318741551|
|  c|     1|0.22314355131420976|
|  c|     0|0.22314355131420976|
+---+------+-------------------+



In [109]:
woe.compute_iv()

{'cat': 0.5924584552018219}